*** Google Colab requierements ***
(Neu chay bang google colab cai dat theo cac lenh ben duoi)


! pip install requests-html

! apt-get update

! apt install chromium-chromedriver

! cp /usr/lib/chromium-browser/chromedriver /usr/bin

! pip install selenium


from selenium import webdriver

options = webdriver.ChromeOptions()

options.add_argument('--headless')

options.add_argument('--no-sandbox')

options.add_argument('--disable-dev-shm-usage')


import json

browser = webdriver.Chrome('chromedriver',options=options)


from requests_html import HTMLSession, HTML

session = HTMLSession()

0. Import modules

In [1]:
####################### Import modules ########################
import json
from requests_html import HTMLSession, HTML
session = HTMLSession()
from selenium import webdriver
browser = webdriver.Chrome(executable_path='./chromedriver/chromedriver')

1. Get link_products

In [ ]:
browser.get('https://www.bestbuy.com/site/laptop-computers/all-laptops/pcmcat138500050001.c?id=pcmcat138500050001&intl=nosplash')

f = open("dataset/product_links.txt",'w')

while True:
    post = HTML(html=browser.page_source)
    head_ = post.find(".sku-item-list")
    link_lists = head_[0].find(".sku-item")
    for i in range(1,len(link_lists)):
        print(link_lists[i])
        sku_title = link_lists[i].find('.sku-title')
        _link = sku_title[0].find('a')
        url = 'https://www.bestbuy.com/'+_link[0].attrs['href']
        f.write(url)
        sku_header = sku_title[0].find('.sku-header')
        if len(sku_header) == 0:
            f.write('\t0')
        f.write('\n')
    next_click_class = post.find('.ficon-caret-right')
    if next_click_class[1].attrs['aria-disabled'] == 'false':
        url_next = next_click_class[1].attrs['href']
        browser.get(url_next)
    else:
        break
f.close()

# Chu y: Trong cac link lays ve co format moi dong la:
# url[\t0]  - [tab0]: san pham co kem theo combo (win/mouse/...)

2. Get attributes

In [ ]:
def getAttributes():
    res = {'Price': ['Price']}
    
    browser.get('https://www.bestbuy.com/site/laptop-computers/all-laptops/pcmcat138500050001.c?id=pcmcat138500050001&intl=nosplash')

    f = open('dataset/product_links.txt', 'r')
    links = f.readlines()
    f.close()

    count = 0
    for link in links:
        count += 1
        url = link[:-1]
        url = url.split('\t')

        if (len(url) == 1):
            browser.get(url[0])
            post = HTML(html=browser.page_source)

            print(count)
            print(url[0])           
            
            specifications = post.find('.spec-categories')
            if len(specifications) != 0:
                #specifications_components = specifications[0].find('.section-title') # cac thanh phan lon
                specifications_components_details = specifications[0].find('.category-wrapper') # Tat ca thanh phan lon va nho
                for component in specifications_components_details:
                    data = component.text.split('\n')
                    key1 = data[0]
                    if key1 not in res.keys():
                        res[key1] = []
                    i = 1
                    while i < len(data):
                        key2 = data[i]
                        if key2 not in res[key1]:
                            res[key1].append(key2)
                        i += 1
                        if data[i] == 'Info':
                            i += 1
                        i += 1
            else:
                continue
    return res

dic = getAttributes()
r = json.dumps(dic)
f = open('dataset/attrs.json', 'w')
f.write(r)
f.close()

3. Crawl Data

In [2]:
class Laptop:
    def __init__(self, attrbs_dict):
        self.data = {}
        for key in list(attrbs_dict.keys()):
            self.data[key] = dict(zip(attrbs_dict[key], [""]*len(attrbs_dict[key])))
        
    def convertDataToJsonStr(self):
        return json.dumps(self.data)

    def convertDataToStrCSV(self):
        res = ''
        for key in self.data.keys():
            res += '\t' + '\t'.join(self.data[key].values())
        return res[1:] + '\n'
    
    def get_keys_list(self):
        keys = []
        for key in self.data.keys():
            #keys.extend(self.data[key].keys())
            for sub_key in self.data[key].keys():
                keys.append(key + '__' + sub_key)
        return keys
          
    def setValue(self, component, field, value):
        self.data[component][field] = value

In [3]:
with open('dataset/attrs.json', 'r') as f:
    attrs = f.read()
attrbs_dict = json.loads(attrs)

In [ ]:
browser = webdriver.Chrome(executable_path='./chromedriver/chromedriver')

browser.get('https://www.bestbuy.com/site/laptop-computers/all-laptops/pcmcat138500050001.c?id=pcmcat138500050001&intl=nosplash')

f = open('dataset/product_links.txt', 'r')
links = f.readlines()
f.close()

g = open('dataset/product_infos.csv', 'w')

temp = Laptop(attrbs_dict)
title = '\t'.join(temp.get_keys_list()) + '\n'
g.write(title)

count = 0
for link in links:
    count += 1
    url = link[:-1]
    url = url.split('\t')
    
    if (len(url) == 1):
        laptop = Laptop(attrbs_dict)
        
        browser.get(url[0])
        post = HTML(html=browser.page_source)
                
        print(count)
        print(url[0])
        print('\n')
        
        price = post.find('.priceView-price')
        if len(price) == 0:
            continue
        price = price[0].find('.priceView-hero-price')
        if len(price) == 0:
            continue
        price = price[0].find('.sr-only')
        if len(price) == 0:
            continue
        laptop.setValue('Price', 'Price', price[0].text)
        
        specifications = post.find('.spec-categories')
        if len(specifications) != 0:
            #specifications_components = specifications[0].find('.section-title') # cac thanh phan lon
            specifications_components_details = specifications[0].find('.category-wrapper') # Tat ca thanh phan lon va nho
            for component in specifications_components_details:
                data = component.text.split('\n')
                key1 = data[0]
                i = 1
                while i < len(data):
                    key2 = data[i]
                    i += 1
                    if data[i] == 'Info':
                        i += 1
                    laptop.setValue(key1, key2, data[i])
                    i += 1
        else:
            continue
    
        g.write(laptop.convertDataToStrCSV())
    
g.close()